# Net Position Analysis: From Net-to-Robot to Robot

This notebook analyzes the relationship between:
1. **Net position from robot's perspective** (sonar-detected distance and angle)
2. **Robot position relative to the net** (navigation-based positioning)

The analysis transforms between these two coordinate systems using CSV data from net detection results and navigation data.

## 1. Import Libraries and Utils

Import necessary libraries and our custom net position analysis utilities.

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from pathlib import Path

# Import our custom net position analysis utilities
from utils.net_position_analysis import (
    load_net_analysis_results,
    load_navigation_data,
    calculate_robot_to_net_position,
    analyze_net_position_consistency,
    create_net_position_visualization,
    run_net_position_analysis
)

# Import configuration
from utils.sonar_config import EXPORTS_DIR_DEFAULT

print("Libraries and utilities imported successfully!")

Libraries and utilities imported successfully!


## 2. Load Net Results from CSV

Load the net analysis results that contain sonar-detected distances and angles to the net from the robot's perspective.

In [2]:
# Configuration - set your target bag name here
TARGET_BAG = "2024-08-20_14-31-29"  # Replace with your actual bag name
EXPORTS_FOLDER = Path(EXPORTS_DIR_DEFAULT)

print(f"Target Bag: {TARGET_BAG}")
print(f"Exports Folder: {EXPORTS_FOLDER}")

# Load net analysis results (sonar detections from robot perspective)
try:
    net_df = load_net_analysis_results(TARGET_BAG, EXPORTS_FOLDER)
    print(f"✓ Loaded {len(net_df)} net analysis frames")
    print(f"Columns: {list(net_df.columns)}")
    print(f"Detection success rate: {net_df['detection_success'].mean()*100:.1f}%")
except FileNotFoundError as e:
    print(f"✗ Error loading net results: {e}")
    print("Make sure you have run the sonar analysis and saved results to CSV first.")

Target Bag: 2024-08-20_14-31-29
Exports Folder: /Volumes/LaCie/SOLAQUA/exports
Loading net analysis results: /Volumes/LaCie/SOLAQUA/exports/outputs/2024-08-20_14-31-29_data_cones_analysis_results.csv
✓ Loaded 1059 net analysis frames
Columns: ['frame_index', 'timestamp', 'distance_pixels', 'angle_degrees', 'distance_meters', 'detection_success', 'tracking_status', 'area', 'aspect_ratio', 'solidity', 'extent', 'ellipse_elongation', 'straightness', 'rect', 'centroid_x', 'centroid_y']
Detection success rate: 100.0%


## 2a. Load Sonar Analysis Results and Navigation Data

Load the CSV files containing sonar distance/angle detections and DVL navigation data for comparison.

In [3]:
# Load sonar distance analysis results - search for matching files
print("Searching for sonar analysis results...")

# Try multiple possible patterns
outputs_dir = EXPORTS_FOLDER / "outputs"
possible_patterns = [
    f"{TARGET_BAG}_analysis_results.csv",           # Exact bag name
    f"*{TARGET_BAG}*analysis_results.csv",          # Bag name anywhere in filename
    f"*{TARGET_BAG}*_cones_analysis_results.csv",   # With _cones suffix
]

df_sonar = None
sonar_csv_path = None

for pattern in possible_patterns:
    matching_files = list(outputs_dir.glob(pattern))
    if matching_files:
        sonar_csv_path = matching_files[0]  # Take first match
        print(f"✓ Found sonar analysis file: {sonar_csv_path.name}")
        break

if sonar_csv_path and sonar_csv_path.exists():
    df_sonar = pd.read_csv(sonar_csv_path)
    print(f"✓ Loaded sonar analysis: {len(df_sonar)} frames")
    print(f"Columns: {list(df_sonar.columns)}")
    print(f"Detection success rate: {df_sonar['detection_success'].mean()*100:.1f}%")
else:
    print(f"✗ No sonar analysis CSV found for bag '{TARGET_BAG}' in {outputs_dir}")
    print("\nTROUBLESHOoting:")
    print("1. Check if you have run distance analysis first:")
    print("   - Run notebook 06: Image Analysis (with save_outputs=True)")
    print("   - OR run DistanceAnalysisEngine.analyze_npz_sequence(save_outputs=True)")
    print("\n2. Check available analysis files:")
    
    # List existing analysis files to help user
    analysis_files = list(outputs_dir.glob("*analysis_results.csv"))
    if analysis_files:
        print("   Available analysis files:")
        for f in analysis_files:
            print(f"     - {f.name}")
        print(f"   Update TARGET_BAG to match one of these files")
    else:
        print("   No analysis results files found - run distance analysis first")
    
    df_sonar = None

Searching for sonar analysis results...
✓ Found sonar analysis file: 2024-08-20_14-31-29_data_cones_analysis_results.csv
✓ Loaded sonar analysis: 1059 frames
Columns: ['frame_index', 'timestamp', 'distance_pixels', 'angle_degrees', 'distance_meters', 'detection_success', 'tracking_status', 'area', 'aspect_ratio', 'solidity', 'extent', 'ellipse_elongation', 'straightness', 'rect', 'centroid_x', 'centroid_y']
Detection success rate: 100.0%


In [4]:
# Load navigation data (DVL) - EXACT same method as video generation
print("Loading DVL data using video generation method...")

# Use the same file pattern as the video generation code
nav_file = EXPORTS_FOLDER / "by_bag" / f"navigation_plane_approximation__{TARGET_BAG}_data.csv"

if nav_file.exists():
    print(f"✓ Found navigation file: {nav_file.name}")
    df_nav = pd.read_csv(nav_file)
    
    # Apply same transformations as video generation code
    df_nav["timestamp"] = pd.to_datetime(df_nav["ts_utc"])
    df_nav = df_nav.sort_values("timestamp")
    
    print(f"✓ Loaded navigation data: {len(df_nav)} records")
    print(f"Columns: {list(df_nav.columns)}")
    
    # Check for the key columns that video generation uses
    key_columns = [c for c in ["NetDistance", "NetPitch", "timestamp"] if c in df_nav.columns]
    print(f"Key DVL columns available: {key_columns}")
    
    # Display sample data focusing on the columns used by video generation
    if 'NetDistance' in df_nav.columns:
        valid_distances = df_nav['NetDistance'].dropna()
        print(f"NetDistance: {len(valid_distances)} valid measurements")
        print(f"  Range: {valid_distances.min():.3f} to {valid_distances.max():.3f} m")
        print(f"  Mean: {valid_distances.mean():.3f} ± {valid_distances.std():.3f} m")
    
    if 'NetPitch' in df_nav.columns:
        valid_pitch = df_nav['NetPitch'].dropna()
        print(f"NetPitch: {len(valid_pitch)} valid measurements")
        print(f"  Range: {np.degrees(valid_pitch.min()):.1f}° to {np.degrees(valid_pitch.max()):.1f}°")
        print(f"  Mean: {np.degrees(valid_pitch.mean()):.1f}° ± {np.degrees(valid_pitch.std()):.1f}°")
    
    print(f"\nSample navigation data (first 3 rows):")
    display_cols = ['timestamp', 'NetDistance', 'NetPitch']
    available_cols = [c for c in display_cols if c in df_nav.columns]
    print(df_nav[available_cols].head(3))
    
else:
    print(f"✗ Navigation file not found: {nav_file}")
    
    # Try the alternative loading method as fallback
    print("Trying alternative loading method...")
    try:
        import utils.net_distance_analysis as sda
        BY_BAG_FOLDER = EXPORTS_FOLDER / "by_bag"
        raw_data, _ = sda.load_all_distance_data_for_bag(TARGET_BAG, BY_BAG_FOLDER)
        
        if raw_data and 'navigation' in raw_data and raw_data['navigation'] is not None:
            df_nav = raw_data['navigation'].copy()
            df_nav['timestamp'] = pd.to_datetime(df_nav['timestamp'], errors='coerce')
            print(f"✓ Loaded via fallback method: {len(df_nav)} records")
        else:
            df_nav = None
            print("✗ No navigation data found via fallback method")
    except Exception as e:
        print(f"✗ Fallback loading failed: {e}")
        df_nav = None

Loading DVL data using video generation method...
✓ Found navigation file: navigation_plane_approximation__2024-08-20_14-31-29_data.csv
✓ Loaded navigation data: 559 records
Columns: ['t', 't_header', 't_bag', 't_src', 'bag', 'bag_file', 'topic', 'NormalDVL', 'Altitude', 'NetDistance', 'NetHeading', 'NetPitch', 'NetLock', 'NetVelocity_u', 'NetVelocity_v', 'NetVelocity_w', '__msgtype__', 't0', 't_rel', 'ts_utc', 'ts_oslo', 'timestamp']
Key DVL columns available: ['NetDistance', 'NetPitch', 'timestamp']
NetDistance: 559 valid measurements
  Range: 0.230 to 3.150 m
  Mean: 1.968 ± 0.346 m
NetPitch: 559 valid measurements
  Range: -37.7° to 47.2°
  Mean: 5.1° ± 12.9°

Sample navigation data (first 3 rows):
                            timestamp  NetDistance  NetPitch
0 2024-08-20 12:31:31.549001932+00:00         2.38   0.34494
1 2024-08-20 12:31:31.751519442+00:00         2.37   0.34790
2 2024-08-20 12:31:31.936527729+00:00         2.40   0.34084


In [5]:
# Verify both datasets are loaded before comparison
if df_sonar is not None and df_nav is not None:
    print("✓ Both datasets loaded successfully")
    print(f"Sonar: {len(df_sonar)} frames, {df_sonar['detection_success'].sum()} successful")
    print(f"Navigation: {len(df_nav)} records")
    
    # Set net_df to df_sonar for compatibility with existing code
    net_df = df_sonar.copy()
    print(f"✓ Set net_df for compatibility")
else:
    print("✗ Cannot proceed - missing sonar or navigation data")
    net_df = None

✓ Both datasets loaded successfully
Sonar: 1059 frames, 1059 successful
Navigation: 559 records
✓ Set net_df for compatibility


In [6]:
# Calculate robot-to-net positions (transform sonar detections to world coordinates)
if net_df is not None and df_nav is not None:
    print("Calculating robot-to-net positions...")
    combined_df = calculate_robot_to_net_position(net_df, df_nav)

    print(f"✓ Generated {len(combined_df)} position calculations")
    print(f"Successful detections: {combined_df['detection_success'].sum()}")

    # Show sample of combined data
    print("\nSample combined data:")
    print(combined_df.head())

    # Summary statistics
    print("\nPosition Summary:")
    print(f"Mean sonar distance: {combined_df['sonar_distance_m'].mean():.2f} m")
    print(f"Mean sonar angle: {combined_df['sonar_angle_deg'].mean():.2f}°")
    print(f"Net position range - X: {combined_df['net_x_world'].min():.2f} to {combined_df['net_x_world'].max():.2f} m")
    print(f"Net position range - Y: {combined_df['net_y_world'].min():.2f} to {combined_df['net_y_world'].max():.2f} m")
else:
    print("✗ Cannot proceed - missing data")
    combined_df = None

Calculating robot-to-net positions...
Timestamp range - Sonar: 2024-08-20 12:31:31.621807575 to 2024-08-20 12:32:39.331712246
Timestamp range - Nav:   2024-08-20 12:31:31.549001932 to 2024-08-20 12:32:38.556199074
Merged 1059 records from 1059 sonar and 559 nav records
✓ Generated 1059 position calculations
Successful detections: 1059

Sample combined data:
                      timestamp  frame_index  detection_success  \
0 2024-08-20 12:31:31.621807575            1               True   
1 2024-08-20 12:31:31.681648731            2               True   
2 2024-08-20 12:31:31.742511034            3               True   
3 2024-08-20 12:31:31.818883181            4               True   
4 2024-08-20 12:31:31.874155045            5               True   

   sonar_distance_m  sonar_angle_deg  robot_x  robot_y  robot_heading  \
0          2.387373       209.268875      0.0      0.0        0.52279   
1          2.390208       209.701591      0.0      0.0        0.51002   
2          2.35566

## 4. Analyze Position Consistency

Analyze the consistency of net position estimates and calculate statistics.

In [7]:
# Analyze consistency of net position estimates
consistency_stats = analyze_net_position_consistency(combined_df)

print("=== POSITION CONSISTENCY ANALYSIS ===")
for key, value in consistency_stats.items():
    if isinstance(value, float):
        print(f"{key}: {value:.3f}")
    else:
        print(f"{key}: {value}")

# Filter to successful detections for detailed analysis
success_df = combined_df[combined_df['detection_success']].copy()
print(f"\nDetailed analysis of {len(success_df)} successful detections:")

if len(success_df) > 0:
    # Calculate position variability
    position_variability = np.sqrt(
        (success_df['net_x_world'] - consistency_stats['net_centroid_x'])**2 +
        (success_df['net_y_world'] - consistency_stats['net_centroid_y'])**2
    )

    print(f"95th percentile position error: {np.percentile(position_variability, 95):.3f} m")
    print(f"Maximum position error: {position_variability.max():.3f} m")

=== POSITION CONSISTENCY ANALYSIS ===
total_frames: 1059
successful_detections: 1059
detection_rate: 100.000
net_centroid_x: -1.931
net_centroid_y: -0.352
position_std_m: 0.389
position_95th_percentile_m: 1.232
sonar_distance_mean: 2.037
sonar_distance_std: 0.327
sonar_angle_std: 15.259

Detailed analysis of 1059 successful detections:
95th percentile position error: 1.232 m
Maximum position error: 2.128 m

Detailed analysis of 1059 successful detections:
95th percentile position error: 1.232 m
Maximum position error: 2.128 m


## 5. Visualize Net Position Over Time

Create focused plots showing how the estimated net position changes over time in X and Y coordinates separately.

In [8]:
# Create focused visualization of net position over time
fig = create_net_position_visualization(combined_df, consistency_stats)

# Display the figure
fig.show()

# Optional: Save the figure as HTML
# fig.write_html(f"{TARGET_BAG}_net_position_over_time.html")
# print(f"Visualization saved as {TARGET_BAG}_net_position_over_time.html")

print(f"Visualization shows {len(combined_df[combined_df['detection_success']])} successful detections")
print(f"Net position centroid: ({consistency_stats['net_centroid_x']:.2f}, {consistency_stats['net_centroid_y']:.2f}) m")

Visualization shows 1059 successful detections
Net position centroid: (-1.93, -0.35) m


In [9]:
# Save complete analysis results to CSV
output_file = EXPORTS_FOLDER / "outputs" / f"{TARGET_BAG}_net_position_analysis_complete.csv"

# Create outputs directory if it doesn't exist
output_file.parent.mkdir(parents=True, exist_ok=True)

# Save the combined dataframe with all analysis results
combined_df.to_csv(output_file, index=False)
print(f"✓ Complete analysis results saved to: {output_file}")

# Also save consistency statistics as a separate summary file
stats_file = EXPORTS_FOLDER / "outputs" / f"{TARGET_BAG}_net_position_stats.json"

import json
with open(stats_file, 'w') as f:
    # Convert numpy types to native Python types for JSON serialization
    json_stats = {k: float(v) if isinstance(v, (np.floating, np.integer)) else v
                  for k, v in consistency_stats.items()}
    json.dump(json_stats, f, indent=2)

print(f"✓ Analysis statistics saved to: {stats_file}")

print("\n=== ANALYSIS COMPLETE ===")
print(f"Processed {len(combined_df)} frames with {consistency_stats['detection_rate']:.1f}% detection rate")
print(f"Net estimated at world coordinates: ({consistency_stats['net_centroid_x']:.2f}, {consistency_stats['net_centroid_y']:.2f}) m")
print(f"Position consistency (STD): {consistency_stats['position_std_m']:.2f} m")

✓ Complete analysis results saved to: /Volumes/LaCie/SOLAQUA/exports/outputs/2024-08-20_14-31-29_net_position_analysis_complete.csv
✓ Analysis statistics saved to: /Volumes/LaCie/SOLAQUA/exports/outputs/2024-08-20_14-31-29_net_position_stats.json

=== ANALYSIS COMPLETE ===
Processed 1059 frames with 100.0% detection rate
Net estimated at world coordinates: (-1.93, -0.35) m
Position consistency (STD): 0.39 m


In [10]:
# Use utility functions for comprehensive position comparison
if df_sonar is not None and df_nav is not None:
    print("=== COMPREHENSIVE POSITION COMPARISON ===")
    print("Using utility functions for frame-by-frame synchronization")
    
    # Import utility functions
    from utils.net_position_analysis import run_complete_position_comparison
    
    # Run complete comparison pipeline
    sync_df, stats, fig = run_complete_position_comparison(
        df_sonar=df_sonar,
        df_nav=df_nav,
        target_bag=TARGET_BAG,
        tolerance_seconds=0.5,
        save_results=True,
        exports_folder=EXPORTS_FOLDER
    )
    
    # Display the visualization
    if len(sync_df) > 0:
        fig.show()
        
        print(f"\nVIDEO VALIDATION:")
        print(f"  Applied 180° correction to sonar angles to match video coordinate system")
        print(f"  - Yellow/Orange lines: DVL NetDistance @ NetPitch → XY position")
        print(f"  - Magenta lines: Sonar distance_meters @ (angle_degrees - 180°) → XY position")
        print(f"  - Frame-by-frame synchronization ensures temporal alignment")
        print(f"\nRESULTS SAVED:")
        print(f"  - Synchronized data: {TARGET_BAG}_position_comparison_sync.csv")
        print(f"  - Statistics: {TARGET_BAG}_position_comparison_stats.json")
    else:
        print("No synchronized data available for visualization")
        
else:
    print("Cannot perform position comparison - missing sonar or DVL data")

=== COMPREHENSIVE POSITION COMPARISON ===
Using utility functions for frame-by-frame synchronization
=== COMPLETE POSITION COMPARISON PIPELINE ===
Synchronizing 1059 sonar frames with DVL data...
✓ Successfully synchronized 1054 frame pairs

=== POSITION COMPARISON RESULTS ===
Synchronized frames: 1054
Mean sync tolerance: 0.036s

ANGLE CORRECTION:
  Raw sonar angles:      190.0 ± 15.3°
  Corrected sonar angles: 10.0 ± 15.3°
  DVL angles:            5.5 ± 13.5°

POSITION COMPARISON:
  DVL net position X:    0.171 ± 0.446 m
  DVL net position Y:    1.946 ± 0.374 m
  Sonar net position X:  0.363 ± 0.583 m
  Sonar net position Y:  1.922 ± 0.257 m

POSITION AGREEMENT:
  X position agreement (<0.5m):     916/1054 (86.9%)
  Y position agreement (<0.5m):     943/1054 (89.5%)
  Total position agreement (<1.0m):  935/1054 (88.7%)
✓ Synchronized data saved to: /Volumes/LaCie/SOLAQUA/exports/outputs/2024-08-20_14-31-29_position_comparison_sync.csv
✓ Statistics saved to: /Volumes/LaCie/SOLAQUA/exp


VIDEO VALIDATION:
  Applied 180° correction to sonar angles to match video coordinate system
  - Yellow/Orange lines: DVL NetDistance @ NetPitch → XY position
  - Magenta lines: Sonar distance_meters @ (angle_degrees - 180°) → XY position
  - Frame-by-frame synchronization ensures temporal alignment

RESULTS SAVED:
  - Synchronized data: 2024-08-20_14-31-29_position_comparison_sync.csv
  - Statistics: 2024-08-20_14-31-29_position_comparison_stats.json
